## Lesson 2 - Webcrawling - Homework

### Sujet

On va s'interesser aux données entreprise disponible sur le site de Reuters.
Besoin métier: Analyser les performances financières des sociétés cotées pour décider d'une stratégie d'investissement.

Je vous demande donc de récupérer les infos suivantes :
* les ventes au quartier à fin décembre 2018
* le prix de l'action et son % de changement au moment du crawling
* le % Shares Owned des investisseurs institutionels
* le dividend yield de la company, le secteur et de l'industrie

pour les sociétés suivantes : Aribus, LVMH et Danone.
Un exemple de page https://www.reuters.com/finance/stocks/financial-highlights/LVMH.PA

In [56]:
import requests
import unittest
import re
import pandas as pd
import numpy as np
import datetime

from urllib.request import urlopen
from bs4 import BeautifulSoup

In [57]:
# Paramètres généraux
# Racine page Reuters
website_prefix = "https://www.reuters.com/finance/stocks/financial-highlights/"
# Code des sociétés
lcompanies = ['LVMH.PA','AIR.PA','DANO.PA']
# excapes chars
bad_chars = '()%'

In [58]:
# Récupérer les KPIs des ventes
def getSales(soup):
    """
    Renvoie une liste de KPIs de vente
    """
    l_sales = []
    td = soup.find(text=re.compile(".*Quarter.*Dec.*", flags=re.DOTALL))
    # Nb Sales
    read = td.next.next
    l_sales.append(int(read.text))
    # Mean
    read = read.next.next.next
    l_sales.append(float(read.text.translate(str.maketrans(bad_chars,"   ")).strip().replace(',','')))
    # Min
    read = read.next.next.next
    l_sales.append(float(read.text.translate(str.maketrans(bad_chars,"   ")).strip().replace(',','')))
    # Max
    read = read.next.next.next
    l_sales.append(float(read.text.translate(str.maketrans(bad_chars,"   ")).strip().replace(',','')))
    return l_sales

In [59]:
# Récupérer la valeur en bourse
def getValue(soup):
    """
    Renvoie une liste de KPIs de valeurs boursières
    """
    l_value = []
    
    # Valeur
    td = soup.find(text=re.compile(".*Quarter.*Dec.*", flags=re.DOTALL))
    share_value = soup.find("div", class_= "sectionQuoteDetail")
    children = share_value.findChildren("span")
    l_value.append(float(children[1].text.translate(str.maketrans(bad_chars,"   ")).strip().replace(',','')))
    l_value.append(children[2].text.translate(str.maketrans(bad_chars,"   ")).strip())
    # children[3].text #heure
    l_value.append(datetime.datetime.today())
    
    #% evolution
    share_evol = soup.find("span", class_= "valueContentPercent")
    children = share_evol.findChildren("span" , recursive=True)
    l_value.append(float(children[0].text.translate(str.maketrans(bad_chars,"   ")).strip().replace(',',''))/100)
    
    return l_value

In [60]:
# Récupérer les KPI sur les dividendes
def getDividend(soup):
    """
    Renvoie une liste d'infos sur les dividendes
    """
    div=[]
    for i in range(3):
        t = soup.find(text=re.compile("Dividend Yield", flags=re.DOTALL))
        val = t.parent.parent.findChildren('td',class_='data')[i].text.translate(str.maketrans(bad_chars,"   ")
                                                          ).strip().replace(',','').replace('-','')
        div.append(np.NaN if val == "" else float(val))
    return div

In [61]:
# Récupérer le % détenu paer les insitutionnels
def getOwnedInst(soup):
    """
    Renvoie les infos sur parts détenues par les instutionnels
    """
    td = soup.findAll(text=re.compile(".*Owned.*Institutions.*", flags=re.DOTALL),recursive="True")
    owned = td[0].next.next.text.translate(str.maketrans(bad_chars,"   ")).strip().replace(',','').replace('-','')
    #print(owned)
    return np.NaN if owned == "" else float(owned)

In [62]:
# Récupérer le nom de la société
def getCompanyName(soup,strCompany):
    """
    Renvoie le nom de la société
    """
    return soup.find("h1").text.replace(strCompany,'').translate(str.maketrans(bad_chars,"   ")).strip()

In [63]:
def getStockLib(soup,strCompany):
    """
    Renvoie le nom de la société
    """
    return soup.find("span", class_="nasdaqChangeHeader"
                    ).text.strip().replace(',','').replace(strCompany + " on",'').strip()

In [64]:
# Recherche des KPI d'une companie
def findCompanyKPIs(strCompany):
    """
    Renvoie une liste contenant l'ensemble des KPIs d'une société
    """
    results = []
    
    url = website_prefix + strCompany
    res = requests.get(url)
    
    res = requests.get(url)
    if res.status_code == 200:
        html_doc =  res.text
        soup = BeautifulSoup(html_doc,"html.parser")
    
        results.append(strCompany[:strCompany.find('.')])
        results.append(getStockLib(soup,strCompany))
        results.append(getCompanyName(soup,strCompany))
        sales = getSales(soup)
        # Sales
        for i in range(4): results.append(sales[i])
        # Dividend group
        divid = getDividend(soup)
        for i in range(3): results.append(divid[i])
        # Owned institutional
        results.append(getOwnedInst(soup))
        # Stock Market Value
        value = getValue(soup)
        for i in range(4): results.append(value[i])
        
    
    return [results]

In [65]:
# Init du dataframe d'accueil des résultats
df = pd.DataFrame([
        ["comp1",  "","comp1", 1, 1., 1., 1., 1., 1., 1., 1., 1., 'EUR', datetime.datetime.today(),1.]
        ], columns=['Company_id','Stock','Company','LQ18_sales_nb', 'LQ18_sales_mean',
               'LQ18_sales_min','LQ18_sales_max','Dividend_Yeld_cie','Dividend_Yeld_indus','Dividend_Yeld_sector',
               'pct_owned_inst','current_value','currency','datetime','current_pct_evol'
               ])
df = df.drop([0])

In [66]:
# Section principale de recherche et affichage des KPI
for cie in lcompanies:
    df2 = pd.DataFrame(findCompanyKPIs(cie),
                   columns=['Company_id','Stock','Company', 'LQ18_sales_nb', 'LQ18_sales_mean', 'LQ18_sales_min', 
                            'LQ18_sales_max','Dividend_Yeld_cie','Dividend_Yeld_indus','Dividend_Yeld_sector','pct_owned_inst','current_value',
                            'currency','datetime','current_pct_evol'])
    df = df.append(df2)

df = df.drop_duplicates(['Company_id'])
df = df.set_index('Company_id')
df.transpose()

Company_id,LVMH,AIR,DANO
Stock,Paris Stock Exchange,Paris Stock Exchange,Paris Stock Exchange
Company,LVMH Moet Hennessy Louis Vuitton SE,Airbus SE,Danone SA
LQ18_sales_nb,3,5,5
LQ18_sales_mean,13667.7,23493,6072.6
LQ18_sales_min,13769,26073.4,6142
LQ18_sales_max,13575,21431,6025
Dividend_Yeld_cie,1.92,1.45,2.9
Dividend_Yeld_indus,1.7,1.34,2.78
Dividend_Yeld_sector,2.6,1.64,2.48
pct_owned_inst,NaN,NaN,NaN
